In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from PIL import Image

/home/supersaiyan/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu

In [3]:
device

device(type='cuda')

In [4]:
from sklearn.model_selection import train_test_split
dataset = ImageFolder("base_dir/train_dir/")
train_data, test_data, train_label, test_label = train_test_split(dataset.imgs, dataset.targets, test_size=0.2, random_state=42)
# dataset = ImageFolder("base_dir/val_dir/")
# _, test_data, _, test_label = train_test_split(dataset.imgs, dataset.targets, test_size=1., random_state=42)

# ImageLoader Class

class ImageLoader(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = self.checkChannel(dataset) # some images are CMYK, Grayscale, check only RGB 
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, item):
        image = Image.open(self.dataset[item][0])
        classCategory = self.dataset[item][1]
        if self.transform:
            image = self.transform(image)
        return image, classCategory
        
    
    def checkChannel(self, dataset):
        datasetRGB = []
        for index in range(len(dataset)):
            if (Image.open(dataset[index][0]).getbands() == ("R", "G", "B")): # Check Channels
                datasetRGB.append(dataset[index])
        return datasetRGB

In [24]:
dataset = ImageFolder("base_dir/val_dir/")
eval1_data, eval2_data, eval1_label, eval2_label = train_test_split(dataset.imgs, dataset.targets, test_size=0.5, random_state=42)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(), # transforms.Normalize([0.5]*3, [0.5]*3)
])
eval1_dataset = ImageLoader(eval1_data, test_transform)
eval2_dataset = ImageLoader(eval2_data, test_transform)
eval1_loader = DataLoader(eval1_dataset, batch_size=16, shuffle=True)
eval2_loader = DataLoader(eval2_dataset, batch_size=16, shuffle=True)

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.RandomHorizontalFlip(), transforms.ToTensor(),# transforms.Normalize([0.5]*3, [0.5]*3)
]) # train transform

test_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(), # transforms.Normalize([0.5]*3, [0.5]*3)
]) # test transform

train_dataset = ImageLoader(train_data, train_transform)
test_dataset = ImageLoader(test_data, test_transform)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
# load pretrain model and modify...
# model = models.EfficientNet(pretrained=True)
model = timm.create_model('efficientnet_b0', pretrained=True, drop_rate=0.2)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = True

# # num_ftrs = model.fc.in_features
# fc = nn.Sequential(OrderedDict([
# 								#  ('fc1', nn.Linear(2048, 1000, bias=True)),
# 							    #  ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								#  ('dropout1', nn.Dropout(0.7)),
#                                  ('fc2', nn.Linear(1280, 512)),
# 								 ('BN2', nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								 ('swish1', Swish()),
# 								 ('dropout2', nn.Dropout(0.5)),
# 								 ('fc3', nn.Linear(512, 128)),
# 								 ('BN3', nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 							     ('swish2', Swish()),
# 								 ('fc4', nn.Linear(128, 3)),
# 								 ('output', nn.Softmax(dim=1))
# 							 ]))
# 

model.classifier = nn.Sequential(model.classifier, nn.Linear(1000, 512), nn.Linear(512, 7), nn.Softmax(dim=1))
for param in model.classifier.parameters():
	param.requires_grad = True

model.to(device)

In [14]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
# load pretrain model and modify...
# model = models.EfficientNet(pretrained=True)
model = models.resnet50(pretrained=True)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = True

num_ftrs = model.fc.in_features
fc = nn.Sequential(OrderedDict([#('fc1', nn.Linear(2048, 1000, bias=True)),
# 							     ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								 ('dropout1', nn.Dropout(0.7)),
#                                  ('fc2', nn.Linear(1000, 512)),
								#  ('BN2', nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								#  ('swish1', Swish()),
								#  ('dropout2', nn.Dropout(0.5)),
								#  ('fc3', nn.Linear(num_ftrs, 128)),
								#  ('BN3', nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
							    #  ('swish2', Swish()),
								 ('fc4', nn.Linear(num_ftrs, 7)),
								 ('output', nn.Softmax(dim=1))
							 ]))


model.fc = fc

model.to(device)

/home/supersaiyan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/supersaiyan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [18]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, predictions = torch.max(output, 1)
            correct += (predictions == y).sum().item()
            test_loss = criterion(output, y)
            
    test_loss /= len(loader.dataset)
    print("Average Loss: ", test_loss, "  Accuracy: ", correct, " / ",
    len(loader.dataset), "  ", int(correct / len(loader.dataset) * 100), "%")
    return correct / len(loader.dataset)

In [21]:
# Train and test

def train(num_epoch, model, patience = 10):
    best_acc = 0
    patience_count = 0
    
    for epoch in range(0, num_epoch):
#         current_loss = 0.0
#         current_corrects = 0
        losses = []
        model.train()
        loop = tqdm(enumerate(train_loader), total=len(train_loader)) # create a progress bar
        for batch_idx, (data, targets) in loop:
            if len(data) != 16:
                break
            data = data.to(device=device)
            targets = targets.to(device=device)
            scores = model(data)
            
            loss = criterion(scores, targets)
            optimizer.zero_grad()
            losses.append(loss)
            loss.backward()
            optimizer.step()
            _, preds = torch.max(scores, 1)
#             current_loss += loss.item() * data.size(0)
#             current_corrects += (preds == targets).sum().item()
#             accuracy = int(current_corrects / len(train_loader.dataset) * 100)
            loop.set_description(f"Epoch {epoch+1}/{num_epoch} process: {int((batch_idx / len(train_loader)) * 100)}")
            loop.set_postfix(loss=loss.data.item())
        
        test_acc = test()
        if test_acc > best_acc:
            best_acc = test_acc
            print('NEW BEST=', best_acc)
            patience_count = 0
            # save model
            torch.save({ 
                        'model_state_dict': model.state_dict(), 
                        'optimizer_state_dict': optimizer.state_dict(), 
                        }, 'bestmodel.pt')
        else:
            patience_count += 1
            if patience_count == patience:
                print('EARLY STOPPING, BEST ACC=', best_acc)
                break
        scheduler.step()

In [22]:
train(200, model) # train

Epoch 1/200 process: 99: 100%|█████████▉| 1928/1929 [03:42<00:00,  8.68it/s, loss=1.54]


Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  5734  /  7714    74 %
NEW BEST= 0.7433238268084003


Epoch 2/200 process: 99: 100%|█████████▉| 1928/1929 [03:38<00:00,  8.84it/s, loss=1.42]


Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  6068  /  7714    78 %
NEW BEST= 0.7866217267306197


Epoch 3/200 process: 99: 100%|█████████▉| 1928/1929 [03:42<00:00,  8.66it/s, loss=1.4] 


...

Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  7320  /  7714    94 %


Epoch 44/200 process: 99: 100%|█████████▉| 1928/1929 [03:34<00:00,  8.99it/s, loss=1.17]


Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  7320  /  7714    94 %


Epoch 45/200 process: 99: 100%|█████████▉| 1928/1929 [03:34<00:00,  8.99it/s, loss=1.17]


Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  7345  /  7714    95 %
EARLY STOPPING, BEST ACC= 0.9525537982888255


## Evaludate

In [23]:
test()

Average Loss:  tensor(0.0002, device='cuda:0')   Accuracy:  7345  /  7714    95 %


0.952164894996111

In [25]:
def eval(loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, predictions = torch.max(output, 1)
            correct += (predictions == y).sum().item()
            test_loss = criterion(output, y)
            
    test_loss /= len(loader.dataset)
    print("Average Loss: ", test_loss, "  Accuracy: ", correct, " / ",
    len(loader.dataset), "  ", int(correct / len(loader.dataset) * 100), "%")
    return correct / len(loader.dataset)

In [26]:
eval(eval1_loader)

Average Loss:  tensor(0.0025, device='cuda:0')   Accuracy:  438  /  469    93 %


0.9339019189765458

In [27]:
eval(eval2_loader)

Average Loss:  tensor(0.0025, device='cuda:0')   Accuracy:  433  /  469    92 %


0.9232409381663113

## Inference

restnet_50 valid: 0.952 test: 0.925

In [30]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
fc = nn.Sequential(OrderedDict([
								 ('fc4', nn.Linear(num_ftrs, 7)),
								 ('output', nn.Softmax(dim=1))
							 ]))
model.fc = fc
print('Built model')

Built model


/home/supersaiyan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/supersaiyan/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [31]:
print("----> Loading checkpoint")
checkpoint = torch.load("./restnet50_92_5.pt") # Try to load last checkpoint
model.load_state_dict(checkpoint["model_state_dict"]) 
model.to(device)
model.eval()
print('loaded')

----> Loading checkpoint
loaded


In [34]:
def RandomImagePrediction(filepath):
    img_array = Image.open(filepath).convert("RGB")
    data_transforms=transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(), 
        # transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    img = data_transforms(img_array).unsqueeze(dim=0) # Returns a new tensor with a dimension of size one inserted at the specified position.
    load = DataLoader(img)
    
    for x in load:
        x=x.to(device)
        pred = model(x)[0].cpu().detach().numpy()
        print(pred)
        return {
            'akiec':pred[0], 
            'bcc': pred[1], 
            'bkl': pred[2], 
            'df': pred[3],
            'mel': pred[4],
            'nv': pred[5],
            'vasc': pred[6]
        }

In [35]:
RandomImagePrediction("./base_dir/val_dir/nv/ISIC_0024327.jpg")

[1.9970610e-31 3.5728524e-24 8.2722826e-24 2.2845349e-26 5.0844127e-21
 1.0000000e+00 1.5727463e-23]


{'akiec': 1.997061e-31,
 'bcc': 3.5728524e-24,
 'bkl': 8.2722826e-24,
 'df': 2.2845349e-26,
 'mel': 5.0844127e-21,
 'nv': 1.0,
 'vasc': 1.5727463e-23}